In [29]:
import sys
import pandas as pd
sys.path.append('..')
from khan_project_data import *

pd.set_option('display.max_rows',None)

file = "/Users/chuaixin/Desktop/pypro/static/git_project.xlsx"
df = pd.read_excel(file)

In [30]:
def self_git(ProjectID):
    """
    Purpose:获取项目内代码库 
    """
    conn = pymysql.connect(**conn_khan)
    cursor = conn.cursor()
    project_git_list = '''  
        SELECT
            PV.project_id '项目ID',
            PV.id '版本ID',
            PR.repository_id '代码库ID',
            lower(GITR.`name`) '代码库名称'
	
        FROM
            kh_project_version PV 
            LEFT JOIN kh_project__repository PR ON PV.project_id = PR.project_id
            LEFT JOIN kh_gitlab_repository GITR ON GITR.id = PR.repository_id

        WHERE
	        pv.project_id = {0} AND GITR.`name` is NOT NULL'''.format(ProjectID)
    
    cursor.execute(project_git_list)
    result = cursor.fetchall()
    # 关闭光标对象
    cursor.close()
    conn.close()
    return result
    
# end def

def depend_git(ProjectID):
    """
    Purpose:获取项目内代码库 
    """
    conn = pymysql.connect(**conn_khan)
    cursor = conn.cursor()
    project_git_list = '''  
        SELECT
            PV.project_id '项目ID',
            PV.id '版本ID',
            PVD.depend_project_id '依赖产品ID',
            P.name '依赖产品名称',
            PR.repository_id '代码库ID',
            lower(GITR.`name`) '代码库名称'      
        FROM
            kh_project_version PV 
            LEFT JOIN kh_project_version_dependency PVD ON PVD.project_version_id = PV.id
            LEFT JOIN kh_project__repository PR ON PVD.depend_project_id = PR.project_id
            LEFT JOIN kh_gitlab_repository GITR ON GITR.id = PR.repository_id
            LEFT JOIN kh_project p ON PVD.depend_project_id = p.id 
        WHERE
            pv.project_id = {0} AND p.status != 0 AND GITR.`name` is NOT NULL
        order by PVD.depend_project_id asc; 
            '''.format(ProjectID)
    
    cursor.execute(project_git_list)
    result = cursor.fetchall()
    # 关闭光标对象
    cursor.close()
    conn.close()
    return result
    
# end def

In [32]:
for pindex, item in df.iterrows():  # looping through row
    show_depend = depend_git(item['项目ID'])    
    print("依赖产品代码库",show_depend)  
    if show_depend:
        df_show_depend = pd.DataFrame(show_depend)  
        print(df_show_depend)
        #统计对df_show_depend进行groupby分组后，可以分多少组，并赋值给dependcount
        dependcount = df_show_depend.groupby(3).ngroups
        df.loc[pindex, '依赖产品数'] = dependcount 
        
         # 通过项目自有代码库比对与依赖产品代码库进行比较，确定匹配数量: 
        if (item['代码库数']):
            df_git = self_git(item['项目ID'])
            df_git_self = pd.DataFrame(df_git)   
            # print('自有代码库：',df_git_self)
            df_depend_match = df_show_depend.loc[df_show_depend[5].isin(df_git_self[3].str.lower()),[3,5]]
            print('匹配结果：',df_depend_match)
            if (df_depend_match.empty):
                df.loc[pindex, '代码库匹配产品数'] = 0
                continue
            dependcount_match = df_depend_match.groupby(3).ngroups
            df.loc[pindex, '代码库匹配产品数'] = dependcount_match
        # end if
    else:
        df.loc[pindex, '依赖产品数'] = '无产品依赖' 

    print("最终输出：",df.loc[pindex, ['项目ID','项目名称','代码库数','依赖产品数','代码库匹配产品数']])   # 添加了这一行，以便查看结果
# # end for

df.to_excel('/Users/chuaixin/Desktop/pypro/static/git_project_result.xlsx')

依赖产品代码库 ((294, 1550, 98, '微服务管控平台', '1141', 'ultra-msa-sso-core'), (294, 1550, 98, '微服务管控平台', '591', 'ultra-uum'), (294, 1550, 98, '微服务管控平台', '629', 'ultra-msa-gateway-admin'), (294, 1550, 98, '微服务管控平台', '630', 'ultra-msa-gateway-api'), (294, 1550, 98, '微服务管控平台', '631', 'ultra-msa-pasm-api'), (294, 1550, 98, '微服务管控平台', '632', 'ultra-msa-sso'), (294, 1550, 98, '微服务管控平台', '633', 'ultra-msa-pasm'), (294, 1550, 98, '微服务管控平台', '635', 'ultra-msa-conf-apollo'), (294, 1550, 98, '微服务管控平台', '1407', 'ultra-mmcp'), (294, 1550, 98, '微服务管控平台', '1609', 'ultra-msa-sso-auth-core'), (294, 1550, 98, '微服务管控平台', '1686', 'ultra-msa-sso-sms-core'), (294, 1550, 98, '微服务管控平台', '2158', 'ultra-loginl'), (294, 1550, 116, '机器人能力中心', '6984', 'ultra-gbot-extend'), (294, 1550, 116, '机器人能力中心', '6834', 'ultra-gbot-report'), (294, 1550, 116, '机器人能力中心', '6835', 'ultra-gbot-knowledge'), (294, 1550, 116, '机器人能力中心', '6836', 'ultra-gbot-engine'), (294, 1550, 116, '机器人能力中心', '642', 'bit-gbot-extend'), (294, 1550, 116, '机器人能力中